In [ ]:
import tensorflow as tf
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
import time
import glob
import os
import re

In [ ]:
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
from google.colab import drive
drive.mount('./mount/')

In [ ]:
buffer_size = 20000
batch_size = 10
epochs = 10
seeds = np.load('mount/My Drive/Colab Notebooks/celeba-dcgan/seeds.npy', allow_pickle=True)

In [ ]:
cross_entropy_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy_loss(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy_loss(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy_loss(tf.ones_like(fake_output), fake_output)

In [ ]:
@tf.function
def training_step(image_batch):
    noise = tf.random.normal([batch_size, 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)
      real_output = discriminator(image_batch, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def training(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()
    for image_batch in dataset:
      training_step(image_batch)
    save_generated(seeds, generator_epochs, epoch + 1)
    generator.save('mount/My Drive/Colab Notebooks/celeba-dcgan/models/generator-' + str(generator_epochs + epoch + 1) + '.h5')
    discriminator.save('mount/My Drive/Colab Notebooks/celeba-dcgan/models/discriminator-' + str(discriminator_epochs + epoch + 1) + '.h5')
    print('Epoch {} time {} minutes'.format(epoch + 1, (time.time()-start) / 60))

In [ ]:
celeba_dataset = np.load('mount/My Drive/Colab Notebooks/celeba-dcgan/celeba-20000.npy', allow_pickle=True)
training_images = celeba_dataset.reshape(celeba_dataset.shape[0], 128, 128, 3).astype('float32')
training_images = (training_images) / 255.0
training_dataset = tf.data.Dataset.from_tensor_slices(training_images).shuffle(buffer_size).batch(batch_size)

In [ ]:
plt.imshow(training_images[0])

In [ ]:
generator_models = glob.glob('mount/My Drive/Colab Notebooks/celeba-dcgan/models/generator-*')
latest_generator_model = max(generator_models, key=os.path.getctime)
generator_epochs = int(re.search(r'\d+', latest_generator_model).group())

In [ ]:
discriminator_models = glob.glob('mount/My Drive/Colab Notebooks/celeba-dcgan/models/discriminator-*')
latest_discriminator_model = max(discriminator_models, key=os.path.getctime)
discriminator_epochs = int(re.search(r'\d+', latest_discriminator_model).group())

In [ ]:
generator = load_model(latest_generator_model)
discriminator = load_model(latest_discriminator_model)

In [ ]:
def save_generated(seeds, generator_epochs, epoch):
  generated_images = generator(seeds, training=False)
  plt.figure(figsize=(10,10))
  for i, image in enumerate(generated_images):
    plt.subplot(5, 5, i + 1)
    plt.imshow(image)
    plt.axis('off')
  plt.savefig('mount/My Drive/Colab Notebooks/celeba-dcgan/progress/generator-' + str(generator_epochs + epoch) + '.png')

In [ ]:
#check before training
latest_generator_model

In [ ]:
latest_discriminator_model

In [ ]:
%%time
training(training_dataset, epochs)

In [ ]:
#check after training
generator_models = glob.glob('mount/My Drive/Colab Notebooks/celeba-dcgan/models/generator-*')
latest_generator_model = max(generator_models, key=os.path.getctime)
latest_generator_model

In [ ]:
discriminator_models = glob.glob('mount/My Drive/Colab Notebooks/celeba-dcgan/models/discriminator-*')
latest_discriminator_model = max(discriminator_models, key=os.path.getctime)
latest_discriminator_model

In [ ]:
noise = tf.random.normal([1, 100])
generated_image_trained = generator(noise, training=False)
plt.imshow(generated_image_trained[0])

In [ ]:
decision = discriminator(generated_image_trained)
print (decision)